In [7]:
import pandas as pd
import os
import sys
import csv
csv.field_size_limit(131072<<4)

In [36]:
inputdir = "/data1/peterr/bilingual_data_for_ABClf/"
outdir = "/data1/peterr/bilingual_data_for_ABClf_output/tsv/"


import parse
def get_domain(url:str) -> str:
    pattern = "{protocol}://{domain}/{rest}"
    p = parse.compile(pattern)
    modified_pattern = "{protocol}://{domain}/"
    mp = parse.compile(modified_pattern)
    try:
        parse_result = p.parse(url)
        domain = parse_result["domain"]
    except TypeError:
        parse_result = mp.parse(url)
        domain = parse_result["domain"]
    return domain
from ABClf import load_lexicon, get_variant, count_variants, counts_to_category
lex = load_lexicon()
cols = "URL_en    URL_is    sent_en    sent_is    bleualign_score    paragraph_id_en    paragraph_id_is    deferred_hash_en    deferred_hash_is          bifixer_hash    bifixer_score    bicleaner_ai_score".split()
for file in os.listdir(inputdir):
    if not file.endswith(".txt"): 
        continue
    print(file)
    p = parse.compile("en-{lang}.deduped.txt")
    parse_results = p.parse(file)
    lang = parse_results["lang"]
    print(lang)

    origdf = pd.read_csv(
        os.path.join(
            inputdir,
            file
        ),
        sep="\t",
        header=None,
        nrows=10
    )
    origdf.columns = [col.replace("_is", f"_{lang}") for col in cols]
    for i, row in origdf.iterrows():
        en_text = row["sent_en"]
        en_domain = get_domain(row["URL_en"])
        en_document_variant = get_variant(en_text)
        break
    row['en_document_level_variant'] = en_document_variant
    row['domain_en'] = en_domain
    row['en_domain_level_variant'] = "UNK"


    outdf = pd.read_csv(
        os.path.join(
            outdir,
            file+".ABClf_output.txt"
        ),
        sep="\t"
    )
    print(f"Shape of existing df: {outdf.shape}")
    first_line = pd.DataFrame(data=row).T
    outdf = pd.concat([first_line, outdf], ignore_index=True).reset_index(drop=True)
    print(f"Shape of corrected df: {outdf.shape}")
    domain_variant = outdf.loc[outdf.domain_en == en_domain, "en_document_level_variant"].mode()[0]
    outdf["en_domain_level_variant"][0] = domain_variant
    outdf.to_csv(
        os.path.join(
            outdir,
            file+".ABClf_output.txt"
        ),
        sep="\t",
        index=False
    )



en-sl.deduped.txt
en-hr.deduped.txt
en-bg.deduped.txt
en-mk.deduped.txt
en-is.deduped.txt
is
Shape of existing df: (2323530, 15)
Shape of corrected df: (2323531, 15)
en-mt.deduped.txt
mt
Shape of existing df: (2585068, 15)
Shape of corrected df: (2585069, 15)
en-tr.deduped.txt
tr
Shape of existing df: (44286136, 15)
Shape of corrected df: (44286137, 15)
